# I/ COVID-19 Data Crawl on Wikipedia
Data on numbers of confirmed, recovered cases, and deaths for COVID-19 in the developing countries is fragmented and not always provided in consistent or machine-friendly formats. Also, in many cases only the latest numbers are available so it's not possible to look at changes over time.

This small Python script will crawl historical data from Wikipedia and export to an Excel file for HAPRI's partners in the respective country to double check. Data from this crawl only serves as reference.

This raw data will be cleaned with STATA in the next step.

For technical issue and further questions, please email Định Nguyễn, DinhNX@ueh.edu.vn.

## Import relevent packages

In [1]:
import os
import requests
import pandas as pd
from openpyxl import load_workbook

## Set working directory

In [2]:
#os.chdir(r'C:\Users\NXDin\Dropbox (Vo Tat Thang)\[0][Master]Database\(1)Library_of_Data\Web-Scrap') # Provide the path here
os.chdir('D:\Dropbox (Vo Tat Thang)\[0][Master]Database\(1)Library_of_Data\Web-Scrap') # Provide the path here

## Set up lists

In [3]:
country_name = ["Cambodia",
                "Laos",
                "Thailand"
               ]
                
country_url = ['https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Cambodia', 
               'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Laos',
               'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Thailand'
              ]
country_table = ['6', #Cambodia: Detail confirmed and recovered cases
                 '1', #Laos: Detail confirmed and recovered cases
                 '11' #Thailand: Death cases
                ]

## Begin crawling

In [4]:
i = 1
for url, name, table in zip(country_url, country_name, country_table):

    df = pd.read_html(url, header=0)[int(table)]

    print("Crawling: {}".format(name))
    
    #Create new excel file for first sheet
    if i == 1:

        writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'), engine = 'openpyxl')
        df.to_excel(writer \
                    , sheet_name = name \
                    , index = False)
        writer.save()
        writer.close() 

    #Append new sheet to the newly created excel file
    elif i != 1: 

        book = load_workbook(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'))
        writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'), engine = 'openpyxl')
        writer.book = book  
        df.to_excel(writer \
                    , sheet_name = name \
                    , index = False)
        writer.save()
        writer.close() 

    else:

        print("Something is wrong")

    i += 1

Crawling: Cambodia
Crawling: Laos
Crawling: Thailand


# Check crawl accuracy

In [5]:
book = load_workbook(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'))
book.sheetnames

['Cambodia', 'Laos', 'Thailand']

In [6]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'), sheet_name='Laos', nrows=10)
display(df)

,Case,Date,Age,Gen⁠der,National⁠ity,Location,Treatment facility,Previous country been to,Status,Note,Source,Unnamed: 11
0,1,24 March 2020,28,Male,Laos,Vientiane,NaN,Thailand,Discharged,NaN,NaN,NaN
1,2,24 March 2020,36,Female,Laos,Vientiane,NaN,No,Discharged,Tour guide,NaN,NaN
2,3,25 March 2020,26,Male,Laos,Vientiane,NaN,Europe,Discharged,Close contact with case 1,NaN,NaN
3,4,26 March 2020,42,Male,Laos,Luang Prabang,NaN,No,Discharged,Driver of same tour group as case 2.,NaN,NaN
4,5,26 March 2020,42,Male,Laos,Luang Prabang,NaN,No,Discharged,NaN,NaN,NaN
5,6,26 March 2020,41,Male,Laos,Vientiane,NaN,No,Discharged,In patient,Close contact with case 3,NaN
6,7,28 March 2020,50,Female,Laos,Luang Prabang,NaN,No,Discharged,In patient,Wife of case 5,NaN
7,8,28 March 2020,18,Male,Laos,Vientiane,NaN,No,Discharged,In patient,Close contact with case 3,NaN
8,9,29 March 2020,22,Female,Laos,Vientiane,NaN,Thailand,Discharged,In patient,Visited her relative in Bangkok,NaN
9,10,25 March 2020,21,Female,Laos,Vientiane,NaN,No,Discharged,In patient,Close contact with case 8,NaN


In [7]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'), sheet_name='Cambodia', nrows=10)
display(df)

,Case,Date confirmed,Age,Gen⁠der,National⁠ity,Detection location,Treatment facility,Previous country been to,Status,Note,Source,Unnamed: 11
0,1,27 January 2020,60,Male,China,Sihanoukville,Preah Si⁠hanouk Referral Hospital,China,Dis⁠charged (10 February),Arrived from Wuhan on 23 January with his family.,[8],NaN
1,2,7 March 2020,38,Male,Cambodia,Siem Reap,Siem Reap Referral Hospital,No,Dis⁠charged (30 March),To have person-to-person spread from his emplo...,[18][91],NaN
2,3,10 March 2020,65,Female,United Kingdom,Kampong Cham,Kampong Cham Pro⁠vincial Hospital,Vietnam,Dis⁠charged (22 March),Case 3-⁠5 were passengers of Viking Cruise Jou...,[19],NaN
3,3,10 March 2020,65,Female,United Kingdom,Kampong Cham,Royal Ph⁠nom Penh Hospital,Vietnam,Dis⁠charged (22 March),Case 3-⁠5 were passengers of Viking Cruise Jou...,[19],NaN
4,4,12 March 2020,73,Male,United Kingdom,Kampong Cham,Khmer-Soviet Friendship Hospital,Vietnam,Dis⁠charged (29 March),Case 3-⁠5 were passengers of Viking Cruise Jou...,[23],NaN
5,5,12 March 2020,69,Female,United Kingdom,Kampong Cham,Khmer-Soviet Friendship Hospital,Vietnam,Dis⁠charged (29 March),Case 3-⁠5 were passengers of Viking Cruise Jou...,[23],NaN
6,6,13 March 2020,49,Male,Canada,Phnom Penh,Khmer-Soviet Friendship Hospital,Thailand,Discharged (18 April),"A staff of Canadian International School, Koh ...",[26],NaN
7,7,13 March 2020,33,Male,Belgium,Phnom Penh,Khmer-Soviet Friendship Hospital,Undisclosed,Dis⁠charged (2 April),Identity requested to be concealed.,[26],NaN
8,8,15 March 2020,35,Male,France,Singapore,Khmer-Soviet Friendship Hospital,Singapore,Dis⁠charged (27 April),Arrived from Singapore on 14 March. Possibly i...,[33][35],NaN
9,9,15 March 2020,4 months,Male,France,Phnom Penh,National Pediatric Hospital,Singapore,Dis⁠charged (3 April),Child of case 8. Spread from his father.,[33][35],NaN


In [8]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_uncleaned.xlsx'), sheet_name='Thailand', nrows=10)
display(df)

,Cases Order,Age,Gender,Province,Nationality,Hospital admitted to,Been to other country,Status,Occupation,Note
0,1,35,Male,Samut Prakan,Thailand,"Bamrasnaradura Infectious Diseases Institute, ...",No,Died on 29 February,Product Consultant at King Power store Sivaree...,Patient had been in contact with many tourists...
1,2,70,Male,Bangkok,Thailand,"Bamrasnaradura Infectious Diseases Institute, ...",No,Died on 23 March,Private car driver,Patient also had tuberculosis.[74][75]
2,3,79,Male,Bangkok,Thailand,"Bamrasnaradura Infectious Diseases Institute, ...",No,Died on 23 March,Muay Thai pundit,Patient had other ailments and severe symptoms...
3,4,45,Male,Bangkok,Thailand,Undisclosed Hospital,No,Died on 23 March,Security guard in Thonglor Pub,"Patient had diabetes and obesity, Admit to hos..."
4,5,50,Male,Narathiwat,Thailand,"Su-ngai Kolok Hospital, Narathiwat Province",Malaysia,Died on 27 March,NaN,The first Thai to die from participation in a ...
5,6,55,Female,Bangkok,Thailand,Undisclosed Hospital,No,Died on 28 March,NaN,Patient had diabetes and hyperlipidemia. patie...
6,7,68,Male,Nonthaburi,Thailand,"Nonthavej Hospital, Nonthaburi Province",No,Died on 28 March,NaN,Patient visited Lumpinee Boxing Stadium on Mar...
7,8,54,Male,Yala,Thailand,Undisclosed Hospital,Malaysia,Died on 29 March,Merchant,"Patient had been in Malaysia on 12 March, admi..."
8,9,56,Female,Bangkok,Thailand,Undisclosed Hospital,No,Died on 29 March,NaN,No underlying disease.
9,10,48,Male,Maha Sarakham,Thailand,"Maha Sarakham Hospital, Maha Sarakam",No,Died on 30 March,Musician,"Patient had diabetes, Intestinal cancer and He..."


## Reference
1. https://realpython.com/openpyxl-excel-spreadsheets-python/
2. https://www.journaldev.com/33306/pandas-read_excel-reading-excel-file-in-python
3. https://stackoverflow.com/questions/12965203/how-to-get-json-from-webpage-into-python-script
4. https://stackoverflow.com/questions/15008970/way-to-read-first-few-lines-for-pandas-dataframe

# II/ Import COVID-19 Data cases from Thailand's Department of Decases Control

The result of this script is the raw COVID-19 cases data from Thailand Department of Decease Control. The data will be cleaned in STATA in the next step.

Data is available at https://data.go.th/en/dataset/covid-19-daily

## Set up URL

In [9]:
data_url = 'https://covid19.th-stat.com/api/open/cases'

## Get data from the server and convert to JSON format

In [10]:
response = requests.get(data_url)
response_json = response.json()
print('Last update: {}'.format(response_json['UpdateDate']))

Last update: 21/06/2020


### Extract data and save to Excel format

In [11]:
data_json = response_json['Data']

In [12]:
writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_Thailand.xlsx'), engine = 'openpyxl')

df1 = pd.DataFrame(data_json)
df1.to_excel(writer, index = False)

writer.save()
writer.close()

## Check crawl accuracy

In [13]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_Thailand.xlsx'), nrows=10)
display(df)

,ConfirmDate,No,Age,Gender,GenderEn,Nation,NationEn,Province,ProvinceId,District,ProvinceEn,Detail,StatQuarantine
0,2020-06-21 00:00:00,3148,6,ชาย,Male,Thailand,NaN,ชลบุรี,9,บางละมุง,Chonburi,NaN,1
1,2020-06-20 00:00:00,3147,28,หญิง,Female,Thailand,NaN,ชลบุรี,9,NaN,Chonburi,NaN,1
2,2020-09-19 00:00:00,3146,26,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
3,2020-09-19 00:00:00,3145,23,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
4,2020-09-19 00:00:00,3144,27,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
5,2020-09-19 00:00:00,3143,23,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
6,2020-09-19 00:00:00,3142,24,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
7,2020-06-18 00:00:00,3141,26,หญิง,Female,Thailand,NaN,ชลบุรี,9,NaN,Chonburi,NaN,1
8,2020-06-18 00:00:00,3140,26,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1
9,2020-06-18 00:00:00,3139,24,ชาย,Male,Thailand,NaN,กรุงเทพมหานคร,1,ราชเทวี,Bangkok,NaN,1


# Import COVID-19 Data cases from covid19japan.com

The result of this script is the raw COVID-19 cases data for Japan. The data will be cleaned in STATA in the next step.

Read more about data source at https://github.com/NXDinh/covid19japan-data#data-sources, https://stopcovid19.metro.tokyo.lg.jp/en/

Data is available at https://github.com/reustle/covid19japan-data

## Set up URL

In [14]:
data_url = 'https://data.covid19japan.com/patient_data/latest.json'

## Get data from the server and convert to JSON format

In [15]:
response = requests.get(data_url)
response_json = response.json()

### Extract data and save to Excel format

In [16]:
data_json = response_json

In [17]:
writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_Japan.xlsx'), engine = 'openpyxl')

df1 = pd.DataFrame(data_json)
df1.to_excel(writer, index = False)

writer.save()
writer.close()

## Check crawl accuracy

In [18]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_Japan.xlsx'), nrows=10)
display(df)

,patientId,dateAnnounced,ageBracket,gender,detectedPrefecture,patientStatus,notes,mhlwPatientNumber,prefecturePatientNumber,prefectureSourceURL,confirmedPatient,residence,sourceURL,relatedPatients,knownCluster,charterFlightPassenger,detectedCityTown,cityPrefectureNumber,deceasedDate,cruisePassengerDisembarked
0,15,2020-01-15,30,M,Kanagawa,Recovered,Recovered (as of 2/28),1,Kanagawa#1,https://www.pref.kanagawa.jp/docs/ga4/bukanshi...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TOK1,2020-01-24,40,M,Tokyo,Recovered,Chinese traveler,2,Tokyo#1,NaN,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TOK2,2020-01-25,30,F,Tokyo,Recovered,Chinese traveler,3,Tokyo#2,NaN,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,2020-01-26,40,M,Aichi,NaN,Chinese traveler,4,Aichi#1,https://www.pref.aichi.jp/uploaded/attachment/...,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,28,NaN,NaN,NaN,NaN,NaN,NaN
4,19,2020-01-28,40,M,Aichi,Hospitalized,Chinese traveler,5,Aichi#2,https://www.pref.aichi.jp/uploaded/attachment/...,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20,2020-01-28,60,M,Nara,NaN,"Bus driver for tourism bus, (with MHLW No.8 No...",6,Nara#1,NaN,True,Nara,https://www.mhlw.go.jp/stf/newpage_09531.html,Osaka#1 Chiba#1,NaN,NaN,NaN,NaN,NaN,NaN
6,HKD1,2020-01-28,40,F,Hokkaido,Discharged,Chinese traveler,7,Hokkaido#1,http://www.pref.hokkaido.lg.jp/hf/kth/kak/hass...,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,OSK1,2020-01-29,40,F,Osaka,Hospitalized,"Travel guide, Reinfected",8 & 148,Osaka#1,NaN,True,Osaka,https://www.mhlw.go.jp/stf/newpage_09531.html,Nara#1,NaN,NaN,NaN,NaN,NaN,NaN
8,1,2020-01-30,50,M,Unspecified,Discharged,Japanese lives in China,9,NaN,NaN,True,"Wuhan, China",https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,Charter Flight,1.0,NaN,NaN,NaN,NaN
9,23,2020-01-30,50,M,Mie,Recovered,"Returned from Wuhan 1/13, Discharged 2/17",10,Mie#1,https://www.pref.mie.lg.jp/YAKUMUS/HP/m0068000...,True,Mie,https://www.mhlw.go.jp/stf/newpage_09531.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Import COVID-19 Data cases from 

https://api.covid19india.org/

## Set up URL

In [19]:
data_url = ['https://api.covid19india.org/raw_data1.json',
            'https://api.covid19india.org/raw_data2.json',
            'https://api.covid19india.org/raw_data3.json',
            'https://api.covid19india.org/raw_data4.json',
            'https://api.covid19india.org/raw_data5.json',
            'https://api.covid19india.org/raw_data6.json'
           ]

## Get data from the server and convert to JSON format

In [20]:
i = 1
for url in data_url:

    response = requests.get(url)
    response_json = response.json()
    data_json = response_json['raw_data']

    print("Crawling: {}".format(url))
    
    #Create new excel file for first sheet
    if i == 1:

        writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_India.xlsx'), engine = 'openpyxl')
        df1 = pd.DataFrame(data_json)
        df1.to_excel(writer \
                    , sheet_name = str(i) \
                    , index = False)
        writer.save()
        writer.close() 

    #Append new sheet to the newly created excel file
    elif i != 1: 

        book = load_workbook(os.path.join("DATARAW", 'COVID19_India.xlsx'))
        writer = pd.ExcelWriter(os.path.join("DATARAW", 'COVID19_India.xlsx'), engine = 'openpyxl')
        writer.book = book
        df1 = pd.DataFrame(data_json)
        df1.to_excel(writer \
                    , sheet_name = str(i) \
                    , index = False)
        writer.save()
        writer.close() 

    else:

        print("Something is wrong")

    i += 1

Crawling: https://api.covid19india.org/raw_data1.json
Crawling: https://api.covid19india.org/raw_data2.json
Crawling: https://api.covid19india.org/raw_data3.json
Crawling: https://api.covid19india.org/raw_data4.json
Crawling: https://api.covid19india.org/raw_data5.json
Crawling: https://api.covid19india.org/raw_data6.json


## Check crawl accuracy

In [21]:
df = pd.read_excel(os.path.join("DATARAW", 'COVID19_India.xlsx'), sheet_name='4', nrows=10)
display(df)

,agebracket,contractedfromwhichpatientsuspected,currentstatus,dateannounced,detectedcity,detecteddistrict,detectedstate,entryid,gender,nationality,notes,numcases,patientnumber,source1,source2,source3,statecode,statepatientnumber,statuschangedate,typeoftransmission
0,NaN,NaN,Hospitalized,10/05/2020,NaN,Ajmer,Rajasthan,10009,NaN,NaN,NaN,2,37900,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
1,NaN,NaN,Hospitalized,10/05/2020,NaN,Dungarpur,Rajasthan,10010,NaN,NaN,NaN,1,37901,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
2,NaN,NaN,Hospitalized,10/05/2020,NaN,Jaipur,Rajasthan,10011,NaN,NaN,NaN,10,37902,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
3,NaN,NaN,Hospitalized,10/05/2020,NaN,Kota,Rajasthan,10012,NaN,NaN,NaN,9,37903,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
4,NaN,NaN,Hospitalized,10/05/2020,NaN,Pali,Rajasthan,10013,NaN,NaN,NaN,2,37904,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
5,NaN,NaN,Hospitalized,10/05/2020,NaN,Udaipur,Rajasthan,10014,NaN,NaN,NaN,9,37905,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
6,NaN,NaN,Deceased,10/05/2020,NaN,Jaipur,Rajasthan,10015,NaN,NaN,NaN,1,37906,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
7,NaN,NaN,Recovered,10/05/2020,NaN,Banswara,Rajasthan,10016,NaN,NaN,NaN,4,37907,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
8,NaN,NaN,Recovered,10/05/2020,NaN,Bharatpur,Rajasthan,10017,NaN,NaN,NaN,2,37908,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
9,NaN,NaN,Recovered,10/05/2020,NaN,Chittorgarh,Rajasthan,10018,NaN,NaN,NaN,6,37909,https://twitter.com/ANI/status/125933138485177...,NaN,NaN,RJ,NaN,NaN,NaN
